# 실습 1 신용카드 이상 탐지

데이터 준비
실습에 사용될 데이터는 Kaggle의 Credit Card Fraud Detection 데이터셋입니다. 이 데이터셋은 거래의 시간, 금액과 함께 28개의 PCA 변환된 특성들을 포함하고 있습니다. 'Class' 레이블은 사기 거래를 나타내는 1과 정상 거래를 나타내는 0으로 구분됩니다.

데이터를 불러오고, 전처리하는 기본적인 코드는 아래와 같습니다:

https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud?resource=download

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# 데이터를 불러옵니다.
credit = pd.read_csv('creditcard.csv')
credit

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [11]:
# target 데이터(class)의 분포를 확인합니다.
print("Class Distribution:")
print(credit['Class'].value_counts())

# 위 모든 feature를 사용해서, class를 예측
data = credit[['Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14',
             'V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
target = credit['Class']

Class Distribution:
Class
0    284315
1       492
Name: count, dtype: int64


In [16]:
# 데이터를 훈련 세트와 테스트 세트로 분할합니다.

훈련용_data, 테스트용_data, 훈련용_target, 테스트용_target = train_test_split(data, target, test_size=0.2, random_state=42, stratify=target)

# 데이터 표준화 작업을 실시합니다,
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(훈련용_data)

표준화_훈련용_data = ss.transform(훈련용_data)
표준화_테스트용_data = ss.transform(테스트용_data)


In [26]:
# 로지스틱 회귀 모델을 생성하고 학습합니다.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# 학습된 모델로 테스트 데이터를 예측하고 평가합니다.
lr = LogisticRegression(C=5, max_iter=200)

lr.fit(표준화_훈련용_data, 훈련용_target)

y_pred = lr.predict(표준화_테스트용_data)

print(classification_report(테스트용_target, y_pred))
print(confusion_matrix(테스트용_target, y_pred))

# 정확도, 정밀도, F1 Score를 계산 및 출력합니다.
print("Accuracy:", accuracy_score(테스트용_target, y_pred))
print("Precision:", precision_score(테스트용_target, y_pred, average='binary'))
print("F1:", f1_score(테스트용_target, y_pred, average='binary'))

# AUC 점수를 계산합니다.
lr_auc_score = roc_auc_score(테스트용_target, lr.predict_proba(표준화_테스트용_data)[:, 1])
print("Logistic Regression AUC score:", lr_auc_score)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.64      0.72        98

    accuracy                           1.00     56962
   macro avg       0.91      0.82      0.86     56962
weighted avg       1.00      1.00      1.00     56962

[[56851    13]
 [   35    63]]
Accuracy: 0.9991573329588147
Precision: 0.8289473684210527
F1: 0.7241379310344828
Logistic Regression AUC score: 0.962136117108633


## Decision Tree 로 직접해보기

In [23]:
# 결정 트리 모델을 생성하고 학습합니다.
dt_model = DecisionTreeClassifier()

dt_model.fit(표준화_훈련용_data, 훈련용_target)
# 학습된 모델로 테스트 데이터를 예측하고 평가합니다.
dt_pred = dt_model.predict(표준화_테스트용_data)
accuracy = accuracy_score(테스트용_target, dt_pred)

print(classification_report(테스트용_target, dt_pred))
print(confusion_matrix(테스트용_target, dt_pred))

print("Accuracy:", accuracy)
print("Precision:", precision_score(테스트용_target, dt_pred, average='binary'))
print("F1:", f1_score(테스트용_target, dt_pred, average='binary'))

# AUC 점수를 계산합니다.
dt_auc_score = roc_auc_score(테스트용_target, dt_model.predict_proba(표준화_테스트용_data)[:, 1])
print("Decision Tree AUC score:", dt_auc_score)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.73      0.75        98

    accuracy                           1.00     56962
   macro avg       0.88      0.87      0.87     56962
weighted avg       1.00      1.00      1.00     56962

[[56841    23]
 [   26    72]]
Accuracy: 0.9991397773954567
Precision: 0.7578947368421053
F1: 0.7461139896373057
Decision Tree AUC score: 0.8671447018593594


## Random Forest 로 해보기

In [24]:
# 랜덤 포레스트 모델을 생성하고 학습합니다.
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

rf.fit(표준화_훈련용_data, 훈련용_target)
# 학습된 모델로 테스트 데이터를 예측하고 평가합니다.
rf_pred = dt_model.predict(표준화_테스트용_data)
accuracy = accuracy_score(테스트용_target, rf_pred)

print(classification_report(테스트용_target, rf_pred))
print(confusion_matrix(테스트용_target, rf_pred))

print("Accuracy:", accuracy)
print("Precision:", precision_score(테스트용_target, rf_pred, average='binary'))
print("F1:", f1_score(테스트용_target, rf_pred, average='binary'))

# AUC 점수를 계산합니다.
rf_auc_score = roc_auc_score(테스트용_target, dt_model.predict_proba(표준화_테스트용_data)[:, 1])
print("Random Forest AUC score:", rf_auc_score)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.76      0.73      0.75        98

    accuracy                           1.00     56962
   macro avg       0.88      0.87      0.87     56962
weighted avg       1.00      1.00      1.00     56962

[[56841    23]
 [   26    72]]
Accuracy: 0.9991397773954567
Precision: 0.7578947368421053
F1: 0.7461139896373057
Random Forest AUC score: 0.8671447018593594


## 퀴즈) SVM 사용해보기

In [9]:
from sklearn.svm import SVR # SVM Regressor -> 회귀 : Regressior 분류 : Classifier
from sklearn.metrics import r2_score, mean_squared_error
# SVM 회귀 모델 정의
svm_regressor = SVR()

# 모델 학습
svm_regressor.fit(표준화_훈련용_data, 훈련용_target)

# 예측
svm_pred = svm_regressor.predict(표준화_테스트용_data)

# 결정 계수 계산
r2 = r2_score(테스트용_target, svm_pred)

# 평균 제곱근 오차 계산
mse = mean_squared_error(테스트용_target, svm_pred)
rmse = np.sqrt(mse)

print("결정 계수:", r2)
print("평균 제곱근 오차:", rmse)

결정 계수: -0.8472782813059108
평균 제곱근 오차: 0.0560388638288309


# 가장 AUC점수가 높았던 모델 GridSearch로 튜닝하기

In [29]:
# 그리드서치 활용하여 하이퍼라파미터 튜닝 - LGB
from sklearn.model_selection import GridSearchCV

# 파라미터 설정
params = {
            'C': [0.01, 0.1, 1, 10],
            'penalty': ['l2'],
            'solver': ['lbfgs'],
            'max_iter': [100, 200, 500],
          }

#그리드 서치 객체 생성
gs = GridSearchCV(LogisticRegression(random_state=50), params, cv=5, n_jobs=-1)
# 학습
gs.fit(표준화_훈련용_data, 훈련용_target)

# 최적의 하이퍼 파라미터 값 출력
print(gs.best_params_)

# 정확도 출력
lr = gs.best_estimator_
print(lr.score(표준화_훈련용_data, 훈련용_target))
print(lr.score(표준화_테스트용_data , 테스트용_target))

{'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.9992275450415853
0.9991397773954567
